### Remove bad trials and plot spectrogram
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_spectrograms_wavelet.html

In [ ]:
import os
import numpy as np
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, outliers_to_nan,crop_empty_data
from ieeg.calc.scaling import rescale
from ieeg.viz.ensemble import chan_grid
from ieeg.timefreq.utils import crop_pad, wavelet_scaleogram
from ieeg.viz.parula import parula_map
save_dir='C:\\Users\\bl314\\Box\\CoganLab\\IndividualMeetings\\Baishen\\ieeg_results\\lexical_delay'

In [ ]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")
subject = subjects[0]
subject_Tag = 'D103'

In [ ]:
raw = raw_from_layout(layout.derivatives['derivatives/clean bchrm'], subject=subject, desc='clean bchrm',extension='.edf',preload=True)
# Choose preload=False to release RAM space (but )
raw = crop_empty_data(raw)

In [ ]:
# Epoches: https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.__getitem__
for epoch, t in zip(
        ("Auditory_stim/CORRECT", "Delay/CORRECT", "Resp/CORRECT"),
        ((-0.5, 1.5), (-0.5, 1.5), (-0.5, 1)),
      ):
    
    # Add time point for timefrequency and crop it
    t1 = t[0] - 0.5
    t2 = t[1] + 0.5
    times = (t1, t2)
    trials = trial_ieeg(raw, epoch, times, preload=True)
    # eeg file is big. If try load multiple subjects. 
    # pointing
    
    # remove bad channels
    outliers_to_nan(trials, outliers=10)
    #Learned from Aaron's sentence rep
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    
    # Multitaper wavelet timefreq
    #freq = np.arange(10, 200., 6.)
    #spectra = spectrogram(raw, freq, 'Auditory_stim', -1.2, 1.2, 'Cue', -0.5, 0,
    #                      n_jobs=-3, verbose=10, time_bandwidth=10, n_cycles=freq/2)
    # verbose: extra messages
    # https://mne.tools/dev/generated/mne.verbose.html#mne.verbose
    #spectra = spectrogram(trials, baseline=baselines, freqs=freq, n_jobs=-3, time_bandwidth=10, n_cycles=freq/2)
    # https://mne.tools/dev/generated/mne.time_frequency.tfr_array_multitaper.html (Notes)
    #crop_pad(spectra, "0.5s")
    freq = np.linspace(0.5, 200, num=80)
    kwargs = dict(average=False, n_jobs=-3, freqs=freq, return_itc=False,
                  n_cycles=freq/2, time_bandwidth=4,
                  # n_fft=int(trials.info['sfreq'] * 2.75),
                  decim=20, )
                  # adaptive=True)
    spectra = trials.compute_tfr(method="multitaper",  **kwargs)
    crop_pad(spectra, "0.5s") # cut the first and final 0.5s, change to zero
    
    # crop the baseline
    if epoch == "Auditory_stim":
        base = spectra.copy().crop(-0.5, 0)
            # Average, and do baseline correction
        base = base.average(lambda x: np.nanmean(x, axis=0), copy=True)
        
    spectra = spectra.average(lambda x: np.nanmean(x, axis=0), copy=True)
    rescale(spectra._data, base._data, mode='ratio', axis=-1)
    
    # Save spectrograms
    #fnames = [os.path.relpath(f, layout.root) for f in good.filenames]
    #spectra.info['subject_info']['files'] = tuple(fnames)
    spectra.info['bads'] = raw.info['bads']
    #https://github.com/coganlab/SentenceRep_analysis/blob/main/analysis/check/multitaper_spec.py
    if not os.path.exists(os.path.join(save_dir, subject)):
        os.mkdir(os.path.join(save_dir, subject))
    filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
    mne.time_frequency.write_tfrs(filename, spectra, overwrite=True)
    #spectra.save(os.path.join(save_dir,subject, 'Auditor-avg.fif'))
    

In [ ]:
# del spectra
epoch =  "CORRECT"#"Delay"# "Auditory_stim"#"CORRECT"#,#
filename = os.path.join(save_dir, subject,f'{epoch}-tfr.h5')
spectra = mne.time_frequency.read_tfrs(filename)
chan_grid(spectra, size = (20,10),vlim=(0.7, 1.4), fmax=200, show=True, cmap=parula_map,)# yscale="log")